# Task 1: Geometric Transformation

In [2]:
%matplotlib tk

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
img = cv2.imread("test_image.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [5]:
plt.imshow(img)
plt.axis('off')
plt.show()

In [6]:
origH, origW = img.shape[:2]
print(origH, origW)

1500 1500


In [7]:
plt.imshow(img)
plt.title("Select the corners manually")
plt.axis("on")
plt.show(block=False)
pts = plt.ginput(4, timeout=0)
plt.close()

In [8]:
# (x,y) of corners
pts = np.array(pts, dtype=np.float32)
print("Selected points:", pts)

Selected points: [[ 156.96753  435.37662]
 [ 891.5455    74.17532]
 [1382.617   1092.8441 ]
 [ 656.1558  1449.987  ]]


In [9]:
import math

In [10]:
# Width and Height of Rectangle
topW = math.sqrt((pts[0][0]-pts[1][0])**2 + (pts[0][1]-pts[1][1])**2)
bottomW = math.sqrt((pts[2][0] - pts[3][0])**2 + (pts[2][1]-pts[3][1])**2)
maxW = int(max(topW, bottomW))

leftH = math.sqrt((pts[0][0]-pts[3][0])**2 + (pts[0][1]-pts[3][1])**2)
rightH = math.sqrt((pts[1][0]-pts[2][0])**2 + (pts[1][1]-pts[2][1])**2)

maxH = int(max(leftH, rightH))

In [11]:
print(maxH, maxW)

1130 818


In [12]:
finalRect = np.array([[0, 0],[maxW-1, 0],[maxW-1, maxH-1],[0, maxH-1]], dtype=np.float32)

In [13]:
delx = pts[3][0] - pts[0][0] # negative if clockwise
dely = pts[3][1] - pts[0][1] # positive

theta = math.atan2(delx, dely)
print(f"Rotation angle in radians: {theta}")
print(f"Rotation angle in degrees: {theta*180/math.pi}")

Rotation angle in radians: 0.4572271042077215
Rotation angle in degrees: 26.197183350090725


In [14]:
cosTheta = abs(np.cos(theta))
sinTheta = abs(np.sin(theta))

newW = int(origH*sinTheta + origW*cosTheta)
newH = int(origH*cosTheta + origW*sinTheta)
print(newH, newW)

2008 2008


In [15]:
padX = (newW-origW)//2
padY = (newH-origH)//2
print(padX, padY)

254 254


In [16]:
# Padded Image before Rotation
paddedImg = np.zeros((newH, newW, 3), dtype=img.dtype)
paddedImg[padY:padY+origH, padX:padX+origW] = img

In [17]:
plt.imshow(paddedImg)
plt.axis('off')
plt.show()

In [18]:
if theta < 0:
    cw = True
else:
    cw = False
print(cw)

False


In [19]:
cosTheta = np.cos(theta)
sinTheta = np.sin(theta)
cx = newW//2
cy = newH//2

tx = cx - (cosTheta*cx-sinTheta*cy)
ty = cy - (sinTheta*cx+cosTheta*cy)

AffineRT =  np.array([
    [cosTheta, -sinTheta, tx],
    [sinTheta, cosTheta, ty],
    [0,0, 1]
])

invAffineRT = np.linalg.inv(AffineRT)[:2,:]

In [20]:
# Rotated image
rotatedImg = np.zeros_like(paddedImg)

# Nearest Neighbour Interpolation of fractional pixels

In [21]:
for j in range(newH):
    for i in range(newW):
        mat = np.dot(invAffineRT, np.array([i,j,1]))
        x = int(round(mat[0])) # Nearest Neighbour
        y = int(round(mat[1]))
        if 0 <= x < newW and 0 <= y < newH:
            rotatedImg[j, i] = paddedImg[y, x]

In [22]:
plt.imshow(rotatedImg)
plt.axis('off')
plt.show()

In [23]:
Shiftedpts = pts + np.array([padX, padY], dtype=np.float32) # after padding
Shiftedpts = np.hstack((Shiftedpts, np.ones((4,1))))
Shiftedpts = np.dot(AffineRT, Shiftedpts.T).T  # (4, 3)

In [24]:
minX = max(int(np.floor(np.min(Shiftedpts[:, 0]))),0)
maxX = min(int(np.ceil(np.max(Shiftedpts[:, 0]))),rotatedImg.shape[1])
minY = max(int(np.floor(np.min(Shiftedpts[:, 1]))),0)
maxY = min(int(np.ceil(np.max(Shiftedpts[:, 1]))),rotatedImg.shape[0])

In [25]:
croppedImg = rotatedImg[minY:maxY, minX:maxX]

In [26]:
plt.imshow(croppedImg)
plt.axis('off')
plt.show()

In [27]:
plt.imsave("outputNN.png", croppedImg)

In [28]:
rotatedImg = np.zeros_like(paddedImg) # restart

# BiLinear Interpolation of fractional pixels

In [29]:
for j in range(newH):
    for i in range(newW):
        mat = np.dot(invAffineRT, np.array([i, j, 1]))
        x = mat[0]
        y = mat[1]

        # Neighbouring pixels
        x0 = int(np.floor(x))
        x1 = x0 + 1
        y0 = int(np.floor(y))
        y1 = y0 + 1

        if 0 <= x0 < newW-1 and 0 <= y0 < newH-1:
            dx = x - x0
            dy = y - y0

            I00 = paddedImg[y0, x0].astype(np.float32)
            I10 = paddedImg[y0, x1].astype(np.float32)
            I01 = paddedImg[y1, x0].astype(np.float32)
            I11 = paddedImg[y1, x1].astype(np.float32)

            BiLi = (1-dy)*((1 -dx)*I00 + dx*I10) + dy*((1-dx)*I01 + dx*I11)

            rotatedImg[j, i] = np.clip(BiLi, 0, 255).astype(np.uint8)


In [30]:
plt.imshow(rotatedImg)
plt.axis('off')
plt.show()

In [31]:
Shiftedpts = pts + np.array([padX, padY], dtype=np.float32) # after padding
Shiftedpts = np.hstack((Shiftedpts, np.ones((4,1))))
Shiftedpts = np.dot(AffineRT, Shiftedpts.T).T  # (4, 3)

In [32]:
minX = max(int(np.floor(np.min(Shiftedpts[:, 0]))),0)
maxX = min(int(np.ceil(np.max(Shiftedpts[:, 0]))),rotatedImg.shape[1])
minY = max(int(np.floor(np.min(Shiftedpts[:, 1]))),0)
maxY = min(int(np.ceil(np.max(Shiftedpts[:, 1]))),rotatedImg.shape[0])

In [33]:
croppedImg = rotatedImg[minY:maxY, minX:maxX]

In [34]:
plt.imshow(croppedImg)
plt.axis('off')
plt.show()

In [35]:
plt.imsave("outputBiLi.png", croppedImg)

In [36]:
# # Scaling
# xscale = maxW/newW
# yscale = maxH/newH

# cx = newW//2
# cy = newH//2

# tx = cx - xscale*cx
# ty = cy - yscale*cy

# AffineS = np.array([
#     [xscale, 0, tx],
#     [0, yscale, ty],
#     [0,0,1]
# ])

# invAffineS = np.linalg.inv(AffineS)[:2,:]

In [37]:
# scaledImg = np.zeros((maxH, maxW, 3), dtype=rotatedImg.dtype)

In [38]:
# for j in range(maxH):
#     for i in range(maxW):
#         mat = np.dot(invAffineS, np.array([i,j,1]))
#         x = int(round(mat[0])) # Nearest Neighbour
#         y = int(round(mat[1]))
#         if 0 <= x < newW and 0 <= y < newH:
#             scaledImg[j, i] = rotatedImg[y, x]

In [39]:
# plt.imshow(scaledImg)
# plt.axis('off') 
# plt.show()